# Resource Graph
[azure-cli-core](https://pypi.org/project/azure-cli-core/) \
[azure.identity](https://pypi.org/project/azure-identity/) \
[azure-mgmt-resource](https://pypi.org/project/azure-mgmt-resource/) \
[azure-mgmt-resourcegraph](https://pypi.org/project/azure-mgmt-resourcegraph/) \
[Resource graph docs](https://learn.microsoft.com/en-us/azure/governance/resource-graph/) \
[resource-graph/samples](https://learn.microsoft.com/en-us/azure/governance/resource-graph/samples/starter?tabs=azure-cli) \

In [ ]:
# Add the Resource Graph library for Python
pip install azure-mgmt-resourcegraph

# Add the Resources library for Python
pip install azure-mgmt-resource

# Add the CLI Core library for Python for authentication (development only!)
pip install azure-cli-core

# Add the Azure identity library for Python
pip install azure.identity

In [ ]:
# Check each installed library
pip show azure-mgmt-resourcegraph azure-mgmt-resource azure-cli-core azure.identity

In [ ]:
from azure.identity import DefaultAzureCredential 
from azure.identity import EnvironmentCredential 
from azure.identity import ClientSecretCredential 

tenant_id = ""
client_id = ""
client_secret = ""

def getCreds ():
    # https://pypi.org/project/azure-identity/#:~:text=library%27s%20credential%20classes.-,DefaultAzureCredential,-DefaultAzureCredential%20is%20appropriate
    
    # recommended!
    # credential = DefaultAzureCredential()
    
    #For Demo purpose...
    # credential = EnvironmentCredential()
    credential = ClientSecretCredential(tenant_id, client_id, client_secret)
    
    return credential

In [ ]:
from azure.mgmt.resource import SubscriptionClient

def getSubsList(credential):
    subsClient = SubscriptionClient(credential)
    
    subsRaw = []
    for sub in subsClient.subscriptions.list():
        subsRaw.append(sub.as_dict())

    subsList = []
    for sub in subsRaw:
        subsList.append(sub.get('subscription_id'))
    return subsList

In [ ]:
# Import Azure Resource Graph library
import azure.mgmt.resourcegraph as arg
import json

# Wrap all the work in a function
def getresources( strQuery,managementgroup=True ):
    # Get your credentials from Azure CLI (development only!) and get your subscription list
    credential = getCreds()

    # Create Azure Resource Graph client and set options
    argClient = arg.ResourceGraphClient(credential)
    argQueryOptions = arg.models.QueryRequestOptions(result_format="objectArray")

    # Create query
    if(managementgroup): 
        # SPN has to have permission on the Management Group
        # Root management Group is always the tenant id
        argQuery = arg.models.QueryRequest(management_groups=[tenant_id], query=strQuery, options=argQueryOptions)
    else: #All subscriptions
        subsList = getSubsList(credential)
        argQuery = arg.models.QueryRequest(subscriptions=subsList, query=strQuery, options=argQueryOptions)

    # Run query
    argResults = argClient.resources(argQuery)

    # Show Python object
    print(json.dumps(argResults.data, indent=2))

getresources("Resources | project name, type | limit 1")

In [ ]:
getresources(f"""

resources
| mv-expand ipConfigurations=properties.ipConfigurations
| mv-expand ip=ipConfigurations.properties.privateIPAddress
| where isnotnull(ip)
| extend virtualMachineID=tostring(properties.virtualMachine.id)
| join kind=leftouter (
    resources
    | where type =~ 'microsoft.compute/virtualmachines'
    | project virtualMachineID = id,VM_Name=name,VM_resourceGroup=resourceGroup,VM_subscriptionId=subscriptionId,VM_type=type)
    on virtualMachineID
| project-away virtualMachineID1
| join kind=leftouter(
    resources
    | project managedBy = id, PEndpoint_Name=name,PEndpoint_resourceGroup=resourceGroup,PEndpoint_subscriptionId=subscriptionId,PEndpoint_type=type)
    on managedBy
| project-away managedBy1
| extend attachedto_id=coalesce(virtualMachineID,managedBy)
| extend attachedto_Name=coalesce(VM_Name,PEndpoint_Name)
| extend attachedto_resourceGroup=coalesce(VM_resourceGroup,PEndpoint_resourceGroup)
| extend attachedto_subscriptionId=coalesce(VM_subscriptionId,PEndpoint_subscriptionId)
| extend attachedto_type=coalesce(VM_type,PEndpoint_type)
| project-away tenantId,kind,properties,sku,plan,tags,identity,zones,extendedLocation,ipConfigurations
| project id,location,name,resourceGroup,subscriptionId,type,attachedto_id,attachedto_Name,attachedto_resourceGroup,attachedto_subscriptionId,attachedto_type
//| project ['id'],type,virtualMachine,properties.ipConfigurations,ip

""",False)

# Graph
[msgraph-sdk](https://pypi.org/project/msgraph-sdk/) \
[azure-cli-core](https://pypi.org/project/azure-cli-core/) \
[azure.identity](https://pypi.org/project/azure-identity/) \
[graph-explorer](https://developer.microsoft.com/en-us/graph/graph-explorer) \



In [ ]:
pip install msgraph-sdk azure-cli-core azure.identity

In [ ]:
from msgraph import GraphServiceClient

def getGraphClient (credential,scopes = ['https://graph.microsoft.com/.default']):
    client = GraphServiceClient(credentials=credential, scopes=scopes)
    return client

![Alt text](image.png)

In [ ]:

credential = getCreds()
client = getGraphClient(credential)

users = await client.users.get()
if users and users.value:
        print(users.value)
        for user in users.value:
            print(user.id, user.display_name, user.mail)

print("--------------------Groups------------------------")
groups = await client.groups.get()
if groups and groups.value:
        print(groups.value)
        for group in groups.value:
            print(group.id, group.display_name)

print("--------------------devices------------------------")
devices = await client.devices.get()
if devices and devices.value:
        print(devices.value)
        for device in devices.value:
            print(device.id, device.display_name)   
